# Aula 06 &mdash; Data Scraping com API do Twitter

Renato Vimieiro
rv2 {em} cin.ufpe.br

março 2017

In [1]:
import pandas as pd
import numpy as np

import tweepy

from functools import reduce
from collections import Counter
import json

## Introdução

Nas aulas anteriores vimos como obter dados diretamente de uma página web. Para isso, utilizamos a biblioteca BeautifulSoup. A biblioteca facilitava a busca de tags e, consequentemente, de informações úteis nas páginas. Embora tenhamos que lançar mão desse recurso em muitas ocasiões, alguns sites fornecem APIs para obtermos dados de forma semi-estruturada. Esses dados possuem formato livre, mas possuem indicadores do conteúdo. As representações mais usuais são [XML](https://en.wikipedia.org/wiki/XML) e [JSON](https://en.wikipedia.org/wiki/JSON).

O formato JSON tem se tornado um dos mais usados para troca de dados na internet. Ele tem substituido inclusive XML. Muitos bancos de dados NoSQL, como MongoDB, fornecem suporte direto a dados nesse formato. Em outras palavras, JSON é, talvez, o formato franco de representação de dados para Data Science. Sendo assim, discutiremos rapidamente sua representação.

## JSON

O formato JSON (lê-se como *jay-son*) é essencialmente um dicionário. Ele descreve uma sequência de pares (chave,valor). Objetos no formato JSON são delimitados por { e }. Por sua vez, os pares são separados por vírgulas. As chaves e valores são separados por : (dois-pontos).

As chaves devem ser sempre strings em unicode. Os valores podem ser:

- string
- número
- objeto
- vetor
- true
- false
- null

Os vetores referenciados acima são como vetores/listas em uma linguagem de programação. Eles podem conter uma sequência de um ou mais valores, incluindo outros vetores e objetos. A gramática do formato pode ser conferida em http://www.json.org/json-pt.html.

### Exemplo de um dado no formato JSON

O exemplo a seguir foi retirado da Wikipedia. Ele ilustra como poderiam ser armazenados diversos dados sobre uma pessoa. A escolha das chaves é completamente arbitrária e definida pelo desenvolvedor.

```json
{
  "firstName": "John",
  "lastName": "Smith",
  "isAlive": true,
  "age": 25,
  "address": {
    "streetAddress": "21 2nd Street",
    "city": "New York",
    "state": "NY",
    "postalCode": "10021-3100"
  },
  "phoneNumbers": [
    {
      "type": "home",
      "number": "212 555-1234"
    },
    {
      "type": "office",
      "number": "646 555-4567"
    },
    {
      "type": "mobile",
      "number": "123 456-7890"
    }
  ],
  "children": [],
  "spouse": null
}
```

## API do Twitter

O Twitter oferece aos desenvolvedores uma API para permitir a interação entre o site e aplicativos de terceiros. A API é uma das ricas entre as redes sociais e, por isso, muitas pesquisas comerciais e científicas usam a plataforma.

A API do Twitter se subdivide em outras duas APIs: REST e Streaming. A API de REST permite aos desenvolvedores acessar os perfis dos usuários (públicos) assim como seus seguidores, listas, timeline, além de efetuar buscas por termos ou usuários na plataforma. A API de Streaming, por outro lado, é dedicada ao monitoramento em tempo real dos tweets sendo postados na plataforma.

Além dessas duas interfaces públicas, o Twitter ainda oferece uma terceira interface comercial, Gnip, para acesso a grandes volumes de dados. O Gnip, basicamente, fornece acesso à base de dados (pública) inteira do Twitter. Enquanto a API de Streaming faz amostragem dos tweets em tempo real (não é muito claro como isso é feito), o Gnip fornece acesso a efetivamente todos as mensagens sendo postadas. O mesmo acontece em relação ao REST, que limita, por exemplo, a busca por termos, não retornando todos os resultados. O Gnip, ao contrário, permite efetuar uma busca em toda a base.

## Acesso à API do Twitter

O acesso à API requer a criação de um perfil na plataforma e registro de um aplicativo. A autenticação é feita através do protocolo [OAuth](https://en.wikipedia.org/wiki/OAuth). A ideia básica do protocolo é que um usuário possa delegar acesso a recursos em um determinado site a terceiros sem revelar suas credenciais. Resumindo, o protocolo permite que chaves de acesso sejam geradas pelo usuário; essas chaves são passadas a um terceiro, a quem se deseja permitir acesso aos recursos do usuário; finalmente, a parte a quem o acesso foi delegado pode conectar-se ao sistema usando as chaves que lhe foram enviadas.

#### Registrando um novo aplicativo

Para registrar um aplicativo e gerar novas chaves de acesso, o usuário deve ir até o site https://apps.twitter.com.

![https://apps.twitter.com](images/tela_inicial_apps_twitter.png)

Após logar, a tela inicial mostrará as aplicações autorizadas no momento.

![https://apps.twitter.com](images/apps_twitter_logado.png)

Para criar/autorizar um novo aplicativo, clique no botão **Create New App**. Em seguida, você será redirecionado para uma página onde deverá preencher os dados do aplicativo

![https://apps.twitter.com](images/create_new_app.png)

e aceitar os termos e condições de uso

![https://apps.twitter.com](images/create_new_app_accept_terms_confirm.png)

Pronto! Agora você já resgistrou o aplicativo e poderá gerar as chaves de acesso como a seguir.

#### Obtendo as chaves de acesso

Após registrar o novo aplicativo, você será redirecionado à página de gerenciamento do aplicativo

![https://apps.twitter.com](images/new_app_twitter.png)

Configure as permissões da maneira como for mais conveniente. Em geral, para apenas minerar os dados, basta permitir que o aplicativo possa somente ler os dados. Isso evita, ou diminui as chances, de postagens indevidas caso alguém obtenha suas chaves.

Para gerar as chaves de acesso, vá até a aba **Keys and Access Tokens**.

![https://apps.twitter.com](images/consumer_key_twitter.png)

Inicialmente o aplicativo não está autorizado a logar em nome do usuário. Você verá que foram assinalados um identificador e uma chave de acesso de usuário (*Consumer key and secret*), como mostrado na figura acima. Essas chaves servem para autenticar o usuário, contudo, elas não autenticam o aplicativo, como mostrado na figura abaixo.

![https://apps.twitter.com](images/authorize_app_twitter.png)

Para autorizar o aplicativo, clique no botão **Create my access tokens**. O Twitter então gerará novos identificadores e chaves de acesso para o aplicativo (*Access token and secret*). Esses deverão ser usados em conjunto com as chaves de usuário para se ter acesso à API.

![https://apps.twitter.com](images/access_tokens_twitter.png)

É importante lembrar que esses dois pares de identificadores e chaves nunca devem estar legíveis em seu programa. Eles são como senhas e, portanto, devem ser protegidos.

Agora que já obtivemos as chaves de acesso, podemos iniciar a coleta de dados.

## Acessando a API com Tweepy

Existem diversas bibliotecas de Python para acessar a API do Twitter, como listado na página dos desenvolvedores (https://dev.twitter.com/resources/twitter-libraries). As duas bibliotecas mais usadas (pelo que vi na internet) são [Python-Twitter](https://github.com/bear/python-twitter) e [Tweepy](https://github.com/tweepy/tweepy). No entanto, Tweepy está melhor documentada e possui melhor suporte à API de Streaming. Assim, usaremos Tweepy em nossos exemplos.

#### Conectando-se a API com Tweepy

A biblioteca Tweepy, na verdade, é um wrapper para os serviços listados na API do Twitter. Ela se encarrega dos detalhes de codificação e requisições ao servidor, abstraindo as consultas através de funções e métodos. A biblioteca abstrai inclusive os detalhes de autenticação.

O primeiro passo para coletar os dados é autenticar o usuário e o aplicativo. Isso é feito da seguinte forma:


In [2]:
# As chaves estao armazenadas em um arquivo de configuracoes
# Python possui uma biblioteca para processar esses arquivos (ver configparser)
with open('../configs.txt') as f:
    chaves = json.load(f)
    
# Autentica o usuario na API
auth = tweepy.OAuthHandler(chaves["consumer_key"], chaves["consumer_secret"])

# Autentica o aplicativo
auth.set_access_token(chaves["access_token"], chaves["access_token_secret"])


O objeto `auth` contém a sessão autenticada e deverá ser usado nas conexões com a API.

#### Acesso à REST API

O acesso à REST API é feito através de objetos da classe `tweepy.API`. Os objetos dessa classe possuem métodos que encapsulam os acessos às funcionalidades da API. Assim, devemos instaciar a classe usando a conexão criada anteriormente. É importante respeitar os limites impostos pela API, caso contrário, podemos ser bloqueados. As requisições que ultrapassarem os limites do Twitter resultaram em exceções do tipo `tweepy.RateLimitError`. Tweepy pode controlar automaticamente esses acessos. Podemos estabelecer que as chamadas a determinados métodos devem esperar até que o limite de requisições seja reestabelecido. O limite de requisições varia dependendo do tipo de requisição. O Twitter também faz constantes modificações nesses limites. Então, você sempre deve consultar a documentação da API para se certificar dos limites atuais. Eles podem ser conferidos em https://dev.twitter.com/rest/public/rate-limits.

In [3]:
api = tweepy.API(auth, wait_on_rate_limit=False)

O objeto `api` agora é nossa interface de acesso ao Twitter.

Podemos usá-la para saber quais são os trending topics nos diversos lugares onde essa informação é monitorada.

In [4]:
api.trends_available()

[{'country': '',
  'countryCode': None,
  'name': 'Worldwide',
  'parentid': 0,
  'placeType': {'code': 19, 'name': 'Supername'},
  'url': 'http://where.yahooapis.com/v1/place/1',
  'woeid': 1},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Winnipeg',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/2972',
  'woeid': 2972},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Ottawa',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/3369',
  'woeid': 3369},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Quebec',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/3444',
  'woeid': 3444},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Montreal',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/3534',

In [6]:
trendtopics = api.trends_place(455824)
#trendtopics = api.trends_place(455821)
trendtopics

[{'as_of': '2017-08-29T16:59:36Z',
  'created_at': '2017-08-29T16:53:52Z',
  'locations': [{'name': 'Recife', 'woeid': 455824}],
  'trends': [{'name': 'III Guerra Mundial',
    'promoted_content': None,
    'query': '%22III+Guerra+Mundial%22',
    'tweet_volume': 92235,
    'url': 'http://twitter.com/search?q=%22III+Guerra+Mundial%22'},
   {'name': '#OProblemaDaMinhaVidaÉ',
    'promoted_content': None,
    'query': '%23OProblemaDaMinhaVida%C3%89',
    'tweet_volume': None,
    'url': 'http://twitter.com/search?q=%23OProblemaDaMinhaVida%C3%89'},
   {'name': '#TercaDetremuraSDV',
    'promoted_content': None,
    'query': '%23TercaDetremuraSDV',
    'tweet_volume': 16989,
    'url': 'http://twitter.com/search?q=%23TercaDetremuraSDV'},
   {'name': 'PARABÉNS LUCERO',
    'promoted_content': None,
    'query': '%22PARAB%C3%89NS+LUCERO%22',
    'tweet_volume': None,
    'url': 'http://twitter.com/search?q=%22PARAB%C3%89NS+LUCERO%22'},
   {'name': '#VisibilidadeLésbica',
    'promoted_conten

Como pode ser visto, o resultado é uma lista de objetos no formato JSON (um dicionário). Podemos usar Pandas para criar um data frame com os dados obtidos. Repare que a requisição retorna um único objeto com quatro chaves: data da apuração, data da requisição, local, e os tópicos. Nossa intenção é criar um data frame com os dados dos tópicos. As outras informações são complementares e podem ser replicadas no data frame se necessário após a criação.

In [7]:
dfTrendTopics = pd.DataFrame(trendtopics[0]['trends'])

In [8]:
dfTrendTopics.head(10)

,name,promoted_content,query,tweet_volume,url
0,III Guerra Mundial,None,%22III+Guerra+Mundial%22,92235.0,http://twitter.com/search?q=%22III+Guerra+Mund...
1,#OProblemaDaMinhaVidaÉ,None,%23OProblemaDaMinhaVida%C3%89,NaN,http://twitter.com/search?q=%23OProblemaDaMinh...
2,#TercaDetremuraSDV,None,%23TercaDetremuraSDV,16989.0,http://twitter.com/search?q=%23TercaDetremuraSDV
3,PARABÉNS LUCERO,None,%22PARAB%C3%89NS+LUCERO%22,NaN,http://twitter.com/search?q=%22PARAB%C3%89NS+L...
4,#VisibilidadeLésbica,None,%23VisibilidadeL%C3%A9sbica,NaN,http://twitter.com/search?q=%23VisibilidadeL%C...
5,#MelhorCoisaEh,None,%23MelhorCoisaEh,NaN,http://twitter.com/search?q=%23MelhorCoisaEh
6,#FlickerSessionsDublin,None,%23FlickerSessionsDublin,10695.0,http://twitter.com/search?q=%23FlickerSessions...
7,Fufuca,None,Fufuca,NaN,http://twitter.com/search?q=Fufuca
8,Kenarik Boujikian,None,%22Kenarik+Boujikian%22,NaN,http://twitter.com/search?q=%22Kenarik+Boujiki...
9,Pasadena,None,Pasadena,10697.0,http://twitter.com/search?q=Pasadena


In [9]:
dfTrendTopics.sort_values(by='tweet_volume',ascending=False).\
                head(10)[['name','tweet_volume']]

,name,tweet_volume
47,#VMAs,392329.0
12,Bear,99228.0
0,III Guerra Mundial,92235.0
28,Van Dijk,52376.0
34,#5HonGMA,41421.0
23,ISAC,28669.0
17,Emre Mor,24936.0
2,#TercaDetremuraSDV,16989.0
11,Kim Jong-un,16925.0
31,Rever,10952.0


Podemos também obter a lista de tweets de um usuário

In [10]:
trumpsTweets = api.user_timeline(screen_name='realdonaldtrump')
trumpsTweets[0]

Status(geo=None, retweet_count=7392, truncated=False, coordinates=None, place=None, contributors=None, retweeted=False, author=User(profile_use_background_image=True, follow_request_sent=False, followers_count=36995477, profile_image_url_https='https://pbs.twimg.com/profile_images/874276197357596672/kUuht00m_normal.jpg', created_at=datetime.datetime(2009, 3, 18, 13, 46, 38), profile_sidebar_fill_color='C5CEC0', protected=False, url=None, entities={'description': {'urls': []}}, screen_name='realDonaldTrump', verified=True, profile_banner_url='https://pbs.twimg.com/profile_banners/25073877/1503378994', description='45th President of the United States of America🇺🇸', id_str='25073877', following=False, profile_background_image_url='http://pbs.twimg.com/profile_background_images/530021613/trump_scotland__43_of_70_cc.jpg', default_profile_image=False, profile_link_color='1B95E0', favourites_count=13, statuses_count=35663, friends_count=45, profile_sidebar_border_color='BDDCAD', name='Donald 

O resultado da chamada é uma lista de objetos da classe `Status`. Inspecionando esse objeto, vemos que ele possui um atributo `_json` que armazena o tweet como um dicionário (JSON).

In [11]:
print(type(trumpsTweets[0]._json))
trumpsTweets[0]._json

<class 'dict'>


{'contributors': None,
 'coordinates': None,
 'created_at': 'Tue Aug 29 16:06:57 +0000 2017',
 'entities': {'hashtags': [],
  'media': [{'display_url': 'pic.twitter.com/sUTyXBzer9',
    'expanded_url': 'https://twitter.com/BrazoriaCounty/status/902539081841827842/photo/1',
    'id': 902539076540047360,
    'id_str': '902539076540047360',
    'indices': [20, 43],
    'media_url': 'http://pbs.twimg.com/media/DIZ2gh0UIAA8jCO.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DIZ2gh0UIAA8jCO.jpg',
    'sizes': {'large': {'h': 845, 'resize': 'fit', 'w': 944},
     'medium': {'h': 845, 'resize': 'fit', 'w': 944},
     'small': {'h': 609, 'resize': 'fit', 'w': 680},
     'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
    'source_status_id': 902539081841827842,
    'source_status_id_str': '902539081841827842',
    'source_user_id': 174752321,
    'source_user_id_str': '174752321',
    'type': 'photo',
    'url': 'https://t.co/sUTyXBzer9'}],
  'symbols': [],
  'urls': [],
  'user_ment

Essa representação reflete o modelo de objetos estabelecido pela API do Twitter. Um tweet possui vários campos, como descrito em https://dev.twitter.com/overview/api/tweets. Nem todos os campos são interessantes para a análise de dados. Por exemplo, o tweet contém informações detalhadas sobre o usuário, como o link para a imagem de seu perfil, o link para a imagem que ele usa de fundo, entre outras coisas. Geralmente, esses dados não são úteis para análise. Assim, devemos ser cuidadosos para criar um data frame somente com os dados que julgarmos importantes.

Por exemplo, podemos selecionar os seguintes campos:

- id
- created_at
- entities (hashtags, urls, user_mentions)
- favorite_count
- in_reply_to_screen_name
- in_reply_to_status_id
- in_reply_to_user_id
- lang
- place (country, name)
- retweet_count
- source
- text
- truncated
- user (screen_name, id)

In [3]:
def filtra_campos(tweet):    
    dic = {a[0] : a[1] for a in tweet.items() if a[0] in ['id',
                                    'created_at',
                                    'favorite_count',
                                    'in_reply_to_screen_name',
                                    'in_reply_to_status_id',
                                    'in_reply_to_user_id',
                                    'lang',
                                    'retweet_count',
                                    'source',
                                    'text',
                                    'truncated']}
    dic.update(tweet['entities'])
    if tweet['place']:
        dic.update({'place_'+a[0] : a[1] for a in tweet['place'].items() \
                    if a[0] in ['country', 'name']})
    dic.update({'user_'+a[0] : a[1] for a in tweet['user'].items() \
                if a[0] in ['screen_name', 'id']})
    return dic
    

In [13]:
filtra_campos(trumpsTweets[0]._json)

{'created_at': 'Tue Aug 29 16:06:57 +0000 2017',
 'favorite_count': 0,
 'hashtags': [],
 'id': 902563229968228353,
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_user_id': None,
 'lang': 'und',
 'media': [{'display_url': 'pic.twitter.com/sUTyXBzer9',
   'expanded_url': 'https://twitter.com/BrazoriaCounty/status/902539081841827842/photo/1',
   'id': 902539076540047360,
   'id_str': '902539076540047360',
   'indices': [20, 43],
   'media_url': 'http://pbs.twimg.com/media/DIZ2gh0UIAA8jCO.jpg',
   'media_url_https': 'https://pbs.twimg.com/media/DIZ2gh0UIAA8jCO.jpg',
   'sizes': {'large': {'h': 845, 'resize': 'fit', 'w': 944},
    'medium': {'h': 845, 'resize': 'fit', 'w': 944},
    'small': {'h': 609, 'resize': 'fit', 'w': 680},
    'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
   'source_status_id': 902539081841827842,
   'source_status_id_str': '902539081841827842',
   'source_user_id': 174752321,
   'source_user_id_str': '174752321',
   'type': 'phot

Podemos aplicar a função `filtra_campos` aos tweets recuperados para extrair os campos desejados.

In [14]:
dfTrumpsTweets = pd.DataFrame(list(map(lambda x: filtra_campos(x._json),trumpsTweets)))
dfTrumpsTweets.head()

,created_at,favorite_count,hashtags,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,media,retweet_count,source,symbols,text,truncated,urls,user_id,user_mentions,user_screen_name
0,Tue Aug 29 16:06:57 +0000 2017,0,[],902563229968228353,None,NaN,NaN,und,[{'expanded_url': 'https://twitter.com/Brazori...,7392,"<a href=""http://twitter.com/download/iphone"" r...",[],RT @BrazoriaCounty: https://t.co/sUTyXBzer9,False,[],25073877,"[{'id': 174752321, 'indices': [3, 18], 'id_str...",realDonaldTrump
1,Tue Aug 29 12:26:55 +0000 2017,39442,[],902507855584092160,None,NaN,NaN,en,NaN,9091,"<a href=""http://twitter.com/download/iphone"" r...",[],.@foxandfriends We are not looking to fill a...,False,[],25073877,"[{'id': 15513604, 'indices': [1, 15], 'id_str'...",realDonaldTrump
2,Tue Aug 29 12:10:30 +0000 2017,84495,[],902503724274331653,None,NaN,NaN,en,NaN,15518,"<a href=""http://twitter.com/download/iphone"" r...",[],Leaving now for Texas!,False,[],25073877,[],realDonaldTrump
3,Tue Aug 29 11:31:46 +0000 2017,0,"[{'indices': [100, 108], 'text': 'thefive'}]",902493977533968384,None,NaN,NaN,en,NaN,4082,"<a href=""http://twitter.com/download/iphone"" r...",[],"RT @TheFive: ""Trump just won on law &amp; orde...",False,[],25073877,"[{'id': 334053327, 'indices': [3, 11], 'id_str...",realDonaldTrump
4,Tue Aug 29 11:22:57 +0000 2017,0,"[{'indices': [16, 35], 'text': 'PhotosFromTheF...",902491758235406346,None,NaN,NaN,en,NaN,3965,"<a href=""http://twitter.com/download/iphone"" r...",[],RT @TXMilitary: #PhotosFromTheField: Aerial ph...,False,[],25073877,"[{'id': 111598798, 'indices': [3, 14], 'id_str...",realDonaldTrump


#### Pesquisando tweets

A API do Twitter fornece ainda acesso à pesquisa de tweets usando termos ou *hashtags*.
A descrição completa de como elaborar termos de busca pode ser encontrada em https://dev.twitter.com/rest/public/search. O acesso é fornecido em Tweepy pelo método [`search`](http://docs.tweepy.org/en/v3.5.0/api.html#API.search) da classe API.

A documentação de Tweepy traz a seguinte descrição

```python
API.search(q[, lang][, locale][, rpp][, page][, since_id][, geocode][, show_user])
```

Contudo, os parâmetros descritos estão desatualizados. A documentação do código traz a lista atual de parâmetros, conforme descrição da API do Twitter. São eles:

```python
'q', 'lang', 'locale', 'since_id', 'geocode',
             'max_id', 'since', 'until', 'result_type', 'count',
              'include_entities', 'from', 'to', 'source'
```

Verifique https://dev.twitter.com/rest/reference/get/search/tweets para ver a descrição de cada um dos parâmetros.


In [15]:
tweetsDataScience = api.search('#datascience',count=100,result_type="recent")
len(tweetsDataScience)

100

In [16]:
tweetsDataScience

[Status(geo=None, metadata={'result_type': 'recent', 'iso_language_code': 'und'}, retweet_count=1, truncated=False, coordinates=None, place=None, possibly_sensitive=False, retweeted=False, author=User(profile_use_background_image=True, follow_request_sent=False, followers_count=1222, profile_image_url_https='https://pbs.twimg.com/profile_images/885191163170312192/-3gkTpKz_normal.jpg', created_at=datetime.datetime(2009, 9, 18, 13, 35, 13), profile_sidebar_fill_color='DDEEF6', protected=False, url='https://t.co/ON9JG0QKLf', entities={'description': {'urls': []}, 'url': {'urls': [{'expanded_url': 'https://www.linkedin.com/in/kjsbedi/', 'url': 'https://t.co/ON9JG0QKLf', 'display_url': 'linkedin.com/in/kjsbedi/', 'indices': [0, 23]}]}}, screen_name='KJSBEDI', verified=False, description='#DigitalTransformation #BlockChain #DataScience #R #DevOps #Cloud #EnterpriseArchitecture #IOT #MobileArchitect #RPA', id_str='75281169', following=False, profile_background_image_url='http://abs.twimg.com/

O resultado novamente é uma lista de objetos do tipo `Status`, a qual podemos processar da mesma forma como fizemos com os tweets do Trump.

In [21]:
dfTweetsDataScience = pd.DataFrame(list(map(lambda x: filtra_campos(x._json),
                                            tweetsDataScience)))
dfTweetsDataScience.head()

,created_at,favorite_count,hashtags,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,media,retweet_count,source,symbols,text,truncated,urls,user_id,user_mentions,user_screen_name
0,Tue Aug 29 17:08:19 +0000 2017,0,"[{'indices': [43, 48], 'text': 'code'}, {'indi...",902578671323680769,None,None,None,und,NaN,1,"<a href=""http://sp1n3.com/"" rel=""nofollow"">KJS...",[],RT @andrewshamlet: https://t.co/KJpeop39WA #co...,False,[{'expanded_url': 'http://www.amazon.com/dp/11...,75281169,"[{'id': 2442313279, 'indices': [3, 17], 'id_st...",KJSBEDI
1,Tue Aug 29 17:08:13 +0000 2017,0,"[{'indices': [32, 44], 'text': 'datascience'}]",902578646866681861,None,None,None,en,NaN,10,"<a href=""http://gaggleamp.com/twit/"" rel=""nofo...",[],RT @DataRobot: You can become a #datascience s...,False,"[{'expanded_url': 'http://bit.ly/2xtn9jl', 'ur...",44542581,"[{'id': 622519917, 'indices': [3, 13], 'id_str...",tweetingAsGreg
2,Tue Aug 29 17:07:41 +0000 2017,0,"[{'indices': [52, 64], 'text': 'DataScience'},...",902578515194859521,None,None,None,en,NaN,0,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",[],xtendly: 'How I Analysed 1.6 Million Articles ...,False,"[{'expanded_url': 'https://buff.ly/2vFqk6p', '...",128739276,[],abiconti
3,Tue Aug 29 17:07:21 +0000 2017,0,"[{'indices': [24, 29], 'text': 'code'}, {'indi...",902578428951629824,None,None,None,und,NaN,1,"<a href=""http://www.andrewshamlet.net"" rel=""no...",[],https://t.co/KJpeop39WA #code #learn #data #da...,False,[{'expanded_url': 'http://www.amazon.com/dp/11...,2442313279,[],andrewshamlet
4,Tue Aug 29 17:07:11 +0000 2017,0,"[{'indices': [99, 111], 'text': 'datascience'}]",902578389906804737,None,None,None,en,NaN,0,"<a href=""http://meetedgar.com"" rel=""nofollow"">...",[],How to predict Churn: When Do Email Recipients...,True,[{'expanded_url': 'https://www.blendo.co/blog/...,2253579049,"[{'id': 2988710754, 'indices': [88, 98], 'id_s...",APIrise


Agora podemos iniciar nosso processamento. Podemos, por exemplo, identificar as mensagens mais interessantes, considerando o número de retweets.

In [22]:
dfTweetsDataScience.retweet_count.describe()

count    100.000000
mean      19.870000
std       38.072938
min        0.000000
25%        0.000000
50%        2.000000
75%       18.250000
max      201.000000
Name: retweet_count, dtype: float64

In [23]:
[a for a in dfTweetsDataScience[\
    dfTweetsDataScience.retweet_count >= 16].sort_values(by='retweet_count')['text']]

['RT @dbi_srl: RT evankirstel by  (evankirstel) 4 barriers to #DigitalTransformation\n#StartUps #SMM #IoT #BigData #Fintech #DataScience #DX\n#…',
 'RT @Rbloggers: July 2017 New Package Picks https://t.co/PsNg0BxSjc #rstats #DataScience',
 'RT @Rbloggers: July 2017 New Package Picks https://t.co/PsNg0BxSjc #rstats #DataScience',
 'RT @MikeQuindazzi: #AI + #CRM? By 2021, @IDC estimates a $1.1 trillion boost global business revenues. #UI #BigData #DataScience https://t.…',
 'RT @MikeTamir: A new #AI algorithm summarizes text amazingly well https://t.co/0Oz9uyVt89 #DeepLearning #MachineLearning #DataScience https…',
 'RT @ExperianDataLab: Why We Need More Women Working in #DataScience [LIVE VIDEO]\nhttps://t.co/Y8kiK8cuMQ\n #AI #IoT #BigData #Analytics #Tec…',
 'RT @gotosell: How big is #BigData at big companies?\n#DataScience #Cloud #Data #Tech #IoT #Mpgvip #Defstar5 #DigitalTransformation #SocialMe…',
 'RT @gotosell: How big is #BigData at big companies?\n#DataScience #Cloud #Data #Tech

Observamos que muitas mensagens se tratam de retweets extra-oficiais (não executadas pelo botão de retweet). Essas mensagens citam diversas usuários. Dessa forma, podemos os tweets para encontrar quais são os usuários mais influentes em data science.

Se verificarmos a descrição do objeto Tweet, veremos que ele possui um atributo user_mentions no campo ***entities***. Esse atributo é preenchido pelo Twitter que entrega uma lista de todos os usuários (objetos do tipo User) contidos no texto. Para o nosso exemplo, podemos nos restringir somente ao login do usuário no perfil (screen_name). Depois, podemos usar o screen_name dos usuários mais influentes para recuperar as informações de seus perfis diretamente da API. 

In [24]:
usuariosInfluentes = dfTweetsDataScience.user_mentions.apply(lambda x: 
                                        [y['screen_name'] for y in x])
usuariosInfluentes

0                                       [andrewshamlet]
1                                           [DataRobot]
2                                                    []
3                                                    []
4                                           [blendoapp]
5                                            [RedPixie]
6                                                    []
7                                           [kdnuggets]
8                                           [MikeTamir]
9                                             [dbi_srl]
10                                        [YarmolukDan]
11                                        [YarmolukDan]
12                                 [MikeQuindazzi, IDC]
13                                           [AndySugs]
14                                        [jblefevre60]
15                                                   []
16                                                   []
17                                              

In [28]:
top10Influentes = Counter(reduce(lambda x, y: x+y,
                                 usuariosInfluentes.values)).most_common(10)
top10Influentes

[('kdnuggets', 5),
 ('gotosell', 4),
 ('BigCloudTeam', 4),
 ('Ronald_vanLoon', 4),
 ('jblefevre60', 3),
 ('ipfconline1', 3),
 ('AndySugs', 3),
 ('KirkDBorne', 3),
 ('GDPRdigest', 2),
 ('MediaHutUK', 2)]

Agora que encontramos os dez usuários mais citados nas mensagens, podemos usar a API para recuperar seus dados de perfil. Essa consulta é feita pela interface `lookup_users`.

In [29]:
descTop10Influentes = api.lookup_users(screen_names=[a[0] for a in top10Influentes])
descTop10Influentes

[User(profile_use_background_image=True, follow_request_sent=False, followers_count=92383, status=Status(geo=None, retweet_count=3, truncated=False, coordinates=None, place=None, possibly_sensitive=False, retweeted=False, in_reply_to_user_id=None, source_url='http://bufferapp.com', entities={'urls': [{'expanded_url': 'https://buff.ly/2vmaP8c', 'url': 'https://t.co/cOZylHmUSl', 'display_url': 'buff.ly/2vmaP8c', 'indices': [80, 103]}], 'symbols': [], 'user_mentions': [], 'media': [{'type': 'photo', 'expanded_url': 'https://twitter.com/kdnuggets/status/902581124182007808/photo/1', 'media_url_https': 'https://pbs.twimg.com/media/DIacv6LUwAAA3ZN.jpg', 'id_str': '902581122218901504', 'url': 'https://t.co/n0ZojDKqIh', 'sizes': {'small': {'resize': 'fit', 'h': 190, 'w': 377}, 'medium': {'resize': 'fit', 'h': 190, 'w': 377}, 'large': {'resize': 'fit', 'h': 190, 'w': 377}, 'thumb': {'resize': 'crop', 'h': 150, 'w': 150}}, 'media_url': 'http://pbs.twimg.com/media/DIacv6LUwAAA3ZN.jpg', 'id': 90258

Os objetos do tipo User possuem diversos campos que não são úteis para nossa análise nesse momento. Assim, antes de criar um data frame com os dados dos usuários, devemos filtrá-los, escolhendo somente aquilo que pode ser interessante. Nesse primeiro momento restringiremos nossa análise ao login, nome, descrição, localização, número de seguidores, o número de pessoas que o usuário segue, e o número de tweets que ele já tuitou.

In [30]:
dfTop10Influentes = pd.DataFrame([{'screen_name':a.screen_name,
                                  'name':a.name,
                                   'description':a.description,
                                   'location':a.location,
                                   'followers_count':a.followers_count,
                                   'friends_count':a.friends_count,
                                   'no_tweets':a.statuses_count
                                  } for a in descTop10Influentes],
                                columns = ['screen_name', 'name', 'description',
                                          'location', 'followers_count', 'friends_count', 
                                           'no_tweets'])
dfTop10Influentes

,screen_name,name,description,location,followers_count,friends_count,no_tweets
0,kdnuggets,KDnuggets,"Covering #Analytics, #BigData, #DataMining, #D...","Brookline, MA, USA",92383,394,40730
1,gotosell,GO TO SELL!,Real estate and Property portal\nListing inter...,"Istanbul, Turkey",338,1292,50
2,BigCloudTeam,Big Cloud,Big thinking recruiters specialising in Big Da...,Worldwide,30346,15552,6289
3,Ronald_vanLoon,Ronald van Loon,Helping data driven companies generating value...,#NL. Also connect on LinkedIn,100181,96070,33067
4,jblefevre60,J-Baptiste Lefevre,#digital #innovation #banque #fintech #disrupt...,"Paris, France",14637,2821,50882
5,ipfconline1,ipfconline,Création de Sites Internet Formation et Consei...,"Marseille, France",66935,64811,83171
6,AndySugs,Andy Sugden,Father ov 3 who supports LFC & likes 2 do odd ...,"Lancs, NW England",2744,4432,264380
7,KirkDBorne,Kirk Borne,"The Principal Data Scientist at @BoozAllen, Ph...",Booz Allen Hamilton,166638,64492,76006
8,GDPRdigest,The GDPR Digest,The General Data Protection Regulation: (EU) 2...,"Zurich, Switzerland",127,3,452
9,MediaHutUK,Media Hut,#Promotionalproducts for your business to get ...,"Nottingham, England",589,678,524


#### Trabalhando com cursores

A API do Twitter fornece mecanismos para que o usuário itere sobre o conjunto de resultados de uma requisição. Ao invés de retornar todos os resultados de uma única vez, a API utiliza o conceito de paginação através de cursores. 

Os resultados são divididos em páginas. Na primeira requisição, assume que a página solicitada é a primeira, usando para isso o parâmetro cursor com valor -1. Junto com os resultados, a API retorna o cursor (índice de página) anterior e o próximo. Na requisição subsequente, o usuário informa o próximo cursor e a API retorna a página seguinte com os outros resultados. O usuário pode, então, iterar sobre mais resultados que o limite imposto em cada requisição.

A biblioteca Tweepy possui uma interface direta para a paginação usada pelo Twitter. O tutorial http://docs.tweepy.org/en/v3.5.0/cursor_tutorial.html detalha como utilizar os cursores. Veremos um exemplo a seguir onde recuperaremos os 1000 tweets mais recentes contendo #datascience.

In [31]:
tweetsDataScience = [tweet for tweet in tweepy.Cursor(api.search, q='#datascience',\
                                  count=100,result_type="recent").items(1000)]
len(tweetsDataScience)

1000

O resultado da pesquisa continua sendo uma lista de objetos do tipo `Status`, os quais podemos processar da mesma forma que fizemos com a consulta direta.

In [32]:
dfTweetsDataScience = pd.DataFrame(list(map(lambda x: filtra_campos(x._json),
                                            tweetsDataScience)))
dfTweetsDataScience.head()

,created_at,favorite_count,hashtags,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,media,place_country,place_name,retweet_count,source,symbols,text,truncated,urls,user_id,user_mentions,user_screen_name
0,Tue Aug 29 17:44:07 +0000 2017,0,"[{'indices': [24, 29], 'text': 'code'}, {'indi...",902587681527521281,None,None,NaN,und,NaN,NaN,NaN,0,"<a href=""http://www.andrewshamlet.net"" rel=""no...",[],https://t.co/0gGdCBBYsI #code #learn #data #da...,False,[{'expanded_url': 'http://www.amazon.com/dp/15...,2442313279,[],andrewshamlet
1,Tue Aug 29 17:43:56 +0000 2017,0,"[{'indices': [39, 51], 'text': 'DataScience'},...",902587635243372546,None,None,NaN,en,[{'expanded_url': 'https://twitter.com/Analyti...,NaN,NaN,2,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],RT @AnalyticsVidhya: Taking up role of #DataSc...,False,"[{'expanded_url': 'http://bit.ly/2vnmByX', 'ur...",3119786447,"[{'id': 2311645130, 'indices': [3, 19], 'id_st...",ChicagoAGroup
2,Tue Aug 29 17:43:38 +0000 2017,0,"[{'indices': [21, 36], 'text': 'designthinking...",902587560811274240,None,None,NaN,en,NaN,NaN,NaN,12,"<a href=""http://twitter.com/download/iphone"" r...",[],RT @evankirstel: Can #designthinking unleash o...,False,[],2815408383,"[{'id': 35203319, 'indices': [3, 15], 'id_str'...",martin_steinman
3,Tue Aug 29 17:43:36 +0000 2017,0,"[{'indices': [49, 65], 'text': 'machinelearnin...",902587551663476736,None,None,NaN,en,NaN,NaN,NaN,105,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],RT @jblefevre60: What are the different types ...,False,[],3119786447,"[{'id': 4374719908, 'indices': [3, 15], 'id_st...",ChicagoAGroup
4,Tue Aug 29 17:43:28 +0000 2017,0,"[{'indices': [23, 35], 'text': 'DataScience'},...",902587520520736769,None,None,NaN,en,NaN,NaN,NaN,109,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],RT @ipfconline1: Brief #DataScience History\n#...,False,[],3119786447,"[{'id': 705539763349164032, 'indices': [3, 15]...",ChicagoAGroup


In [33]:
dfTweetsDataScience[['favorite_count','retweet_count']].describe()

,favorite_count,retweet_count
count,1000.000000,1000.000000
mean,0.217000,27.481000
std,1.389907,37.936062
min,0.000000,0.000000
25%,0.000000,1.000000
50%,0.000000,10.000000
75%,0.000000,39.000000
max,29.000000,237.000000


In [34]:
top10Influentes = Counter(reduce(lambda x, y: x+y,
                                 dfTweetsDataScience.user_mentions.apply(lambda x: 
                                        [y['screen_name'] for y in x]).values)).most_common(10)
top10Influentes

[('gp_pulipaka', 85),
 ('KirkDBorne', 53),
 ('kdnuggets', 49),
 ('pinakinariwala', 48),
 ('paulalbright', 30),
 ('ipfconline1', 24),
 ('evankirstel', 23),
 ('cloudpreacher', 22),
 ('Ronald_vanLoon', 21),
 ('jblefevre60', 20)]

In [35]:
dfTop10Influentes = pd.DataFrame([{'screen_name':a.screen_name,
                                  'name':a.name,
                                   'description':a.description,
                                   'location':a.location,
                                   'followers_count':a.followers_count,
                                   'friends_count':a.friends_count,
                                   'no_tweets':a.statuses_count
                                  } \
                                  for a in api.lookup_users(screen_names=[a[0] \
                                                                          for a in top10Influentes])],
                                columns = ['screen_name', 'name', 'description',
                                          'location', 'followers_count', 'friends_count', 
                                           'no_tweets'])
dfTop10Influentes

,screen_name,name,description,location,followers_count,friends_count,no_tweets
0,gp_pulipaka,Dr. GP Pulipaka,Ganapathi Pulipaka | Founder and CEO @deepsing...,"Los Angeles, CA",29098,16322,33305
1,KirkDBorne,Kirk Borne,"The Principal Data Scientist at @BoozAllen, Ph...",Booz Allen Hamilton,166632,64492,76006
2,kdnuggets,KDnuggets,"Covering #Analytics, #BigData, #DataMining, #D...","Brookline, MA, USA",92384,394,40731
3,pinakinariwala,Pinakin Ariwala,"Project Manager @marutitech, sharing data scie...",Ahmedabad,453,140,8316
4,paulalbright,Paul Albright,"Leadership, growth, and getting things done at...","Silicon Valley, CA",7355,4271,5979
5,ipfconline1,ipfconline,Création de Sites Internet Formation et Consei...,"Marseille, France",66931,64797,83172
6,evankirstel,Evan Kirstel,#Influencer #ThoughtLeader Helping #B2B client...,"Boston, MA",141111,137632,548280
7,cloudpreacher,David Holm,"Futurist, CloudEvangelist, Influencer, Entrepr...","Oslo, Norway",8026,6338,3408
8,Ronald_vanLoon,Ronald van Loon,Helping data driven companies generating value...,#NL. Also connect on LinkedIn,100173,96070,33068
9,jblefevre60,J-Baptiste Lefevre,#digital #innovation #banque #fintech #disrupt...,"Paris, France",14634,2822,50882


## Usando a API de Streaming

Como foi dito anteriormente, a API REST não foi desenvolvida para a coleta de dados em tempo real. De acordo com a documentação da API (https://dev.twitter.com/rest/reference/get/search/tweets), o mecanismo de busca não foi desenvolvido para servir como base para coleta exaustiva de tweets. Veja o que diz a documentação

    Please note that Twitter’s search service and, by extension, the Search API is not meant to be an exhaustive source of Tweets. Not all Tweets will be indexed or made available via the search interface.
    
Caso seja importante coletar um grande volume de dados para processá-los depois, deve-se usar a [interface/API de streaming](https://dev.twitter.com/streaming/overview). Nesse caso, uma conexão é mantida com o servidor do Twitter por um tempo mais longo. Novos tweets serão enviados ao usuário à medida em que são postados.

Mais uma vez, Tweepy fornece os mecanismos necessários para coletas da API de streaming. Em suma, o usuário deve criar um *listener* que executará as ações correspondentes quando um evento ocorrer. Esse listerner é passado para Tweepy que se encarrega de conectar com a API do Twitter.

In [99]:
from tweepy.streaming import StreamListener
from tweepy import Stream

A classe abaixo define um exemplo de um listener que exibe as mensagens na saída padrão sempre que um novo dado é enviado.

In [100]:
class StdOutListener(StreamListener):

    def on_data(self, data):
        print(data)
        return True

    def on_error(self, status):
        print(status)


O segundo passo para se coletar dados da API de streaming é instanciar um objeto `Stream` que será responsável pela conexão. Devemos passar para o construtor a sessão autenticada da mesma forma que na API REST, e um listener.

In [101]:
stream = Stream(auth, StdOutListener())

Na maior parte dos casos, nossa intenção é em coletar apenas tweets específicos de um assunto. Assim como na interface de busca, podemos informar os termos que estamos interessados no parâmetro **track**. Uma descrição mais completa dos parâmetros está disponível em https://dev.twitter.com/streaming/overview/request-parameters

In [ ]:
stream.filter(track=['vamosport' ,'vamosanta', 
                     'classicodasmultidoes', 
                     'sportxsantacruz', 'sport recife'])